In [1]:
import pandas as pd
import numpy as np
import datetime
import gtfstk as gt
import os, zipfile, requests, io, shutil
import multiprocessing as mp
from IPython.display import clear_output
%matplotlib inline

In [2]:
folder=r"N:\Planning - New File Structure\Performance\FY19 OTP\July_1_July_13_1_5\CSV"

In [3]:
def readOnlineFeed(url):
    r=requests.get(url)
    z=zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('x')
    feed=gt.read_gtfs('x',dist_units='mi')
    shutil.rmtree('x')
    return feed
gtfs=readOnlineFeed(r"http://valleyregionaltransit.org/gtfs/VRT_Transit1.zip")

In [19]:
def ToDOTP(folder,gtfs=readOnlineFeed(r"http://valleyregionaltransit.org/gtfs/VRT_Transit1.zip"),save_file=None,route=None):
    feed=gtfs.stop_times.merge(gtfs.stops).merge(gtfs.trips).merge(gtfs.routes)
    feed['Route Name']=feed['route_short_name']+" "+feed['route_long_name']
    df=pd.DataFrame()
    for i in os.listdir(folder):
        df=df.append(pd.read_csv(folder+"\\"+i,parse_dates=['ScheduledStopTime']))
    df=df.merge(feed,
                left_on=['StopName','TripName','StopOrder','MasterRouteName'],
                right_on=['stop_name','trip_id','stop_sequence','route_long_name'],
                how='left')
    df['Count']=1
    df['Early']=df['MinutesEarly']>0
    df['On Time']=(df['MinutesEarly']==0)&(df['MinutesLate']==0)
    df['Late']=df['MinutesLate']>0
    df['Hour']=df['ScheduledStopTime'].dt.hour
    outdf=pd.DataFrame(columns=['Early','On Time','Late','Count'])
    group_cols=['MasterRouteName','TripName','StopName','StopOrder','Hour']
    outdf['Early']=df.groupby(group_cols)['Early'].sum()
    outdf['On Time']=df.groupby(group_cols)['On Time'].sum()
    outdf['Late']=df.groupby(group_cols)['Late'].sum()
    outdf['Count']=df.groupby(group_cols)['Count'].sum()
    outdf.reset_index(inplace=True)
    outdf.sort_values(['MasterRouteName','TripName','StopOrder','StopName','Hour'],inplace=True)
    outdf=outdf.groupby(['MasterRouteName','StopName','Hour'],as_index=False,sort=False).sum()
    outdf['Early']=outdf['Early']/outdf['Count']
    outdf['On Time']=outdf['On Time']/outdf['Count']
    outdf['Late']=outdf['Late']/outdf['Count']
    outdf=outdf[['MasterRouteName','StopName','Hour','Early','On Time','Late']]
    outdf.rename({'MasterRouteName':'Route Name','StopName':'Stop Name'},inplace=True)
    if save_file:
        outdf.to_excel(save_file,index=False)
    return outdf
ToDOTP(folder)

,MasterRouteName,StopName,Hour,Early,On Time,Late
0,BSU Express,CWI Main Campus,9,0.000000,0.611111,0.388889
1,BSU Express,9th & River SWC,9,0.000000,0.000000,1.000000
2,BSU Express,University & Joyce SEC,9,0.000000,0.388889,0.611111
3,BSU Express,Front & 3rd NEC,9,0.000000,0.222222,0.777778
4,BSU Express,Front & 9th NEC,9,0.000000,0.222222,0.777778
5,BSU Express,CWI Main Campus,10,0.000000,1.000000,0.000000
6,BSU Express,9th & River SWC,10,0.000000,1.000000,0.000000
7,BSU Express,University & Joyce SEC,10,0.000000,1.000000,0.000000
8,BSU Express,Front & 3rd NEC,10,0.000000,0.888889,0.111111
9,BSU Express,Front & 9th NEC,10,0.000000,1.000000,0.000000
